In [1]:
# library imports
import numpy as np
import astropy
from astropy.coordinates import SkyCoord, GCRS, ICRS, HeliocentricMeanEcliptic, HeliocentricTrueEcliptic, EarthLocation
from astropy.units import deg, au, meter

#  MSE imports
import astro_utils
import asteroid_integrate
import asteroid_data
from asteroid_data import make_data_one_file, get_earth_pos

In [2]:
# RA and DEC of Ceres at 58600.0 according to JPL
ra_deg = 252.310439454
dec_deg = -17.000452272
delta_au = 1.871749452

# Heliocentric coordinates of Ceres
ast_hel_lon_deg = 240.131491
ast_hel_lat_deg = 3.690709
ast_r_au = 2.738311922451

# Heliocentric coordinates of Earth
earth_hel_lon_deg = 215.7084   
earth_hel_lat_deg = 0.0017  
earth_r_au = 1.006212009867

# MJD of observation time
obstime_mjd = 58600.0

In [3]:
# RA and DEC of Ceres at 58600.5 according to JPL
ra_deg = 252.250075738
dec_deg = -17.009538673
delta_au = 1.86800464519883

# Heliocentric coordinates of Ceres
ast_hel_lon_deg = 240.239114
ast_hel_lat_deg = 3.671885
ast_r_au = 2.738704926

# Heliocentric coordinates of Earth
earth_hel_lon_deg = 216.1950
earth_hel_lat_deg = 0.0017  
earth_r_au = 1.006347536397

# MJD of observation time
obstime_mjd = 58600.5

In [4]:
# Reference frames
# https://docs.astropy.org/en/stable/api/astropy.coordinates.GCRS.html
# https://docs.astropy.org/en/stable/api/astropy.coordinates.ICRS.html
frame_earth = GCRS
frame_solar = HeliocentricMeanEcliptic

In [5]:
# Earth location of the Palomar observatory
geoloc_palomar_auto = EarthLocation.of_site('Palomar')
print(f'geolocation of Palomar observatory:')
print(f'geoloc_palomar = {geoloc_palomar_auto}')
print(f'geoloc_palomar geodetic = {geoloc_palomar_auto.geodetic}')

geolocation of Palomar observatory:
geoloc_palomar = (-2410346.78217658, -4758666.82504051, 3487942.97502457) m
geoloc_palomar geodetic = GeodeticLocation(lon=<Longitude -116.863 deg>, lat=<Latitude 33.356 deg>, height=<Quantity 1706. m>)


In [6]:
# For some reason, the 'nice' way to do this below leads to a bug in constructor to GCRS
# geoloc_palomar = EarthLocation(-2410346.8, -4758666.8, 3487943, unit='m')

# Workaround: extract the geolocation of the Palomar observatory using EarthLocation.of_site
geoloc_palomar = [-2410346.78217658, -4758666.82504051, 3487942.97502457] * meter

**Coordinates of Ceres at MJD 58600.5 as SkyCoord instance**

In [7]:
# c = SkyCoord(ra=ra_deg, dec=dec_deg, unit='deg', frame=frame_earth)

In [8]:
# c = SkyCoord(ra=ra_deg*astropy.units.deg, dec=dec_deg*astropy.units.deg, distance=delta_au*astropy.units.au, frame=frame_earth)

In [9]:
# c = SkyCoord(ra=ra_deg, dec=dec_deg, distance=delta_au, unit='deg', frame=frame_earth)

In [10]:
# switch from numbers to class instances

# RA and DEC of asteroid in earth frame
ra = ra_deg * deg
dec = dec_deg * deg
delta = delta_au * au

# Asteroid in solar frame
ast_hel_lon = ast_hel_lon_deg * deg
ast_hel_lat = ast_hel_lat_deg * deg
ast_r = ast_r_au * au

# Earth in solar frame
earth_hel_lon = earth_hel_lon_deg * deg
earth_hel_lat = earth_hel_lat_deg * deg
earth_r = earth_r_au * au

# distances of zero and one AU
zero_au = 0.0 * au 
one_au = 1.0 * au

# angle of zero degrees
zero_deg = 0.0 * deg

# observation time
obstime = astropy.time.Time(val=obstime_mjd, format='mjd')

# display class outputs
print(f'ra      = {ra}')
print(f'dec     = {dec}')
print(f'delta   = {delta}')
print(f'obstime = {repr(obstime)}')

ra      = 252.250075738 deg
dec     = -17.009538673 deg
delta   = 1.86800464519883 AU
obstime = <Time object: scale='utc' format='mjd' value=58600.5>


In [11]:
# Coordinates of Ceres in geocentric frame; include observation time & location.  
# Use correct distance delta_au from JPL
# ast_geo = GCRS(ra=ra, dec=dec, distance=distance, obstime=obstime, obsgeoloc=geoloc_palomar)
# ast_geo = GCRS(ra=ra, dec=dec, distance=delta, obstime=obstime)
ast_geo = SkyCoord(ra=ra, dec=dec, distance=delta, obstime=obstime, frame=frame_earth)
ast_geo

<SkyCoord (GCRS: obstime=58600.5, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (ra, dec, distance) in (deg, deg, AU)
    (252.25007574, -17.00953867, 1.86800465)>

In [12]:
# Coordinates of Earth in geocentric frame; this is easy, distance is zero!
# earth_geo = GCRS(ra=zero_deg, dec=zero_deg, distance=zero_au, obstime=obstime, obsgeoloc = geoloc_palomar)
earth_geo = SkyCoord(ra=zero_deg, dec=zero_deg, distance=zero_au, obstime=obstime, frame=frame_earth)
earth_geo

<SkyCoord (GCRS: obstime=58600.5, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (ra, dec, distance) in (deg, deg, AU)
    (0., 0., 0.)>

**Unit direction of Ceres in Heliocentric (J2000) Frame from RA and Dec**

In [13]:
# Relative displacement from Earth to Ceres in the Heliocentric frame
# This is critical, need to transform from ICRS to Heliocentric for axes to be oriented correctly
ast_hel = ast_geo.transform_to(frame_solar)
earth_hel = earth_geo.transform_to(frame_solar)
rel_hel = (ast_hel.cartesian - earth_hel.cartesian)
print(f'Heliocentric Cartesian coordinates from JPL RA / DEC Data:')
print('Ceres:   ', ast_hel.cartesian)
print('Earth:   ', earth_hel.cartesian)
print('Relative:', rel_hel)

Heliocentric Cartesian coordinates from JPL RA / DEC Data:
Ceres:    (-1.35182396, -2.37951197, 0.17540461) AU
Earth:    (-0.80710993, -0.60131852, 3.06973525e-05) AU
Relative: (-0.54471404, -1.77819345, 0.17537391) AU


In [14]:
# Extract arrays from heliocentric objects
ast_jpl = ast_hel.cartesian.xyz.value
earth_jpl = earth_hel.cartesian.xyz.value
rel_jpl = rel_hel.xyz.value

In [15]:
# Direction from earth to asteroid
u_jpl = rel_jpl / np.linalg.norm(rel_jpl)
print('Direction from Earth to Ceres:')
np.round(u_jpl, 6)

Direction from Earth to Ceres:


array([-0.291602, -0.951921,  0.093883])

In [16]:
# Calculation in a single step using coordinate transformations.
# don't need the distance, just the RA and the Dec!
ast_geo2 = SkyCoord(ra=ra, dec=dec, distance=one_au, obstime=obstime, frame=GCRS)
u_jpl2 = ast_geo2.transform_to(frame_solar).cartesian.xyz.value
u_jpl2 = u_jpl2 / np.linalg.norm(u_jpl2)
norm_diff = np.linalg.norm(u_jpl2 - u_jpl)
print(f'Difference between u_jpl and u_jpl2 method:')
print(f'Norm of difference =  {norm_diff:8.6f} = {np.rad2deg(norm_diff):5.3f} degrees')

Difference between u_jpl and u_jpl2 method:
Norm of difference =  0.319307 = 18.295 degrees


For some reason (not sure why) the direct transformation in one step isn't matching the explicit procedure.
The results above are way off.

In [17]:
# Calculation in a single step using coordinate transformations.
# don't need the distance, just the RA and the Dec!
ast_geo2 = SkyCoord(ra=ra, dec=dec, distance=one_au, obstime=obstime, frame=ICRS)
u_jpl2 = ast_geo2.transform_to(frame_solar).cartesian.xyz.value
u_jpl2 = u_jpl2 / np.linalg.norm(u_jpl2)
np.round(u_jpl2, 6)

array([-0.287882, -0.953112,  0.093281])

Changing the frame from GCRS to ICRS in this part gets it much closer, but it's still a bit off.

In [18]:
norm_diff = np.linalg.norm(u_jpl2 - u_jpl)
print(f'Difference between u_jpl and u_jpl2 method:')
print(f'Norm of difference =  {norm_diff:8.6f} = {np.rad2deg(norm_diff):5.3f} degrees')

Difference between u_jpl and u_jpl2 method:
Norm of difference =  0.003952 = 0.226 degrees


**Unit Direction of Ceres Using Heliocentric Latitude & Longitude from JPL**

In [19]:
# Heliocentric coordinates of Ceres and Earth
ast_hel2 = SkyCoord(lon=ast_hel_lon, lat=ast_hel_lat, distance=ast_r, obstime=obstime, frame=frame_solar)
earth_hel2 = SkyCoord(lon=earth_hel_lon, lat=earth_hel_lat, distance=earth_r, obstime=obstime, frame=frame_solar)

# Cartesian vectors
ast_jpl2 = ast_hel2.cartesian.xyz.value
earth_jpl2 = earth_hel2.cartesian.xyz.value
rel_jpl2 = ast_jpl2 - earth_jpl2
u_jpl2 = rel_jpl2 / np.linalg.norm(rel_jpl2)

In [20]:
print(f'Heliocentric Cartesian coordinates from JPL Heliocentric Coordinate Data:')
print('Ceres:    ', ast_hel2.cartesian)
print('Earth:    ', earth_hel2.cartesian)
print('Relative: ', rel_jpl2)
print('Direction:', u_jpl2)

Heliocentric Cartesian coordinates from JPL Heliocentric Coordinate Data:
Ceres:     (-1.35665161, -2.37260154, 0.17539385) AU
Earth:     (-0.81213439, -0.59428369, 2.98589325e-05) AU
Relative:  [-0.54451722 -1.77831785  0.17536399]
Direction: [-0.29148734 -0.95195729  0.09387469]


In [21]:
ast_jpl

array([-1.35182396, -2.37951197,  0.17540461])

In [22]:
ast_jpl2

array([-1.35665161, -2.37260154,  0.17539385])

**Compare two methods of Using JPL Data to Generate Directions**

In [23]:
# Compare two methods 
diff_ast = ast_jpl2 - ast_jpl
diff_earth = earth_jpl2 - earth_jpl
diff_rel = rel_jpl2 - rel_jpl
diff_u = u_jpl2 - u_jpl
diff_u_norm = np.linalg.norm(diff_u)

print(f'Difference in position (heliocentric long/lat - RA/DEC)')
print(f'Asteroid  : norm={np.linalg.norm(diff_ast):5.2e} AU')
print(f'Earth     : norm={np.linalg.norm(diff_earth):5.2e} AU')
print(f'Rel       : norm={np.linalg.norm(diff_rel):5.2e} AU')
print(f'Direction : norm={diff_u_norm:5.2e} / {np.rad2deg(diff_u_norm):6.4f} degrees')
print(f'Asteroid: ', diff_ast)
print('Earth    :', diff_earth)
print('Rel      :', diff_rel)

Difference in position (heliocentric long/lat - RA/DEC)
Asteroid  : norm=8.43e-03 AU
Earth     : norm=8.64e-03 AU
Rel       : norm=2.33e-04 AU
Direction : norm=1.21e-04 / 0.0069 degrees
Asteroid:  [-4.82764399e-03  6.91043089e-03 -1.07572755e-05]
Earth    : [-5.02445987e-03  7.03483064e-03 -8.38420036e-07]
Rel      : [ 1.96815885e-04 -1.24399748e-04 -9.91885545e-06]


In [24]:
ast_hel2.cartesian - ast_hel.cartesian

<CartesianRepresentation (x, y, z) in AU
    (-0.00482764, 0.00691043, -1.07572755e-05)>

**MSE Integrated Coordinates of Ceres at 58600.5**

In [25]:
ast_elt = asteroid_integrate.load_data()

In [26]:
inputs, outputs = make_data_one_file(0, 1000)

In [27]:
inputs.keys()

dict_keys(['a', 'e', 'inc', 'Omega', 'omega', 'f', 'epoch', 'ts'])

In [28]:
outputs.keys()

dict_keys(['q', 'v', 'u'])

In [29]:
astro_utils.mjd_to_date(int(obstime_mjd))

datetime.date(2019, 4, 27)

In [30]:
astro_utils.mjd_to_datetime(obstime_mjd)

datetime.datetime(2019, 4, 27, 12, 0)

In [31]:
ast_elt

,Num,Name,epoch_mjd,a,e,inc,Omega,omega,M,H,G,Ref,f,P,n,long,theta,pomega,T_peri
Num,,,,,,,,,,,,,,,,,,,
1,1,Ceres,58600.0,2.769165,0.076009,0.184901,1.401596,1.284522,1.350398,3.34,0.12,JPL 46,1.501306,1683.145749,0.003733,4.036516,4.187424,2.686118,-361.745873
2,2,Pallas,58600.0,2.772466,0.230337,0.608007,3.020817,5.411373,1.041946,4.13,0.11,JPL 35,1.490912,1686.155979,0.003726,3.190951,3.639917,2.149005,-279.616804
3,3,Juno,58600.0,2.669150,0.256942,0.226699,2.964490,4.330836,0.609557,5.33,0.32,JPL 108,0.996719,1592.787270,0.003945,1.621697,2.008860,1.012141,-154.522558
4,4,Vesta,58600.0,2.361418,0.088721,0.124647,1.811840,2.630709,1.673106,3.20,0.32,JPL 34,-4.436417,1325.432768,0.004740,6.115656,0.006132,4.442550,-352.940421
5,5,Astraea,58600.0,2.574249,0.191095,0.093672,2.470978,6.260280,4.928221,6.85,0.15,JPL 108,-1.738676,1508.600442,0.004165,1.093108,0.709396,2.448072,325.328481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541124,541124,2018 RP23,58600.0,2.586399,0.289358,0.088749,2.000720,3.913328,1.075531,17.30,0.15,JPL 7,1.654537,1519.293350,0.004136,0.706394,1.285400,-0.369137,-260.066715
541125,541125,2018 RV23,58600.0,3.113036,0.213678,0.203046,0.544794,0.242079,0.130760,16.10,0.15,JPL 8,0.206083,2006.201725,0.003132,0.917633,0.992956,0.786873,-41.751113
541126,541126,2018 RP24,58600.0,2.453880,0.176693,0.194504,2.649626,3.695880,0.937231,17.30,0.15,JPL 6,1.258854,1404.036362,0.004475,0.999551,1.321174,0.062320,-209.433076


In [77]:
# Index for Ceres
idx_ast = np.where(ast_elt.Name=='Ceres')

# Get the index for observation time
ts = inputs['ts'][idx_ast].flatten()

# Correct horrible mistake where the MJD timestamps in MSE file are all too low by 0.5
ts = ts + 0.5

In [78]:
ts

array([51544.5, 51545.5, 51546.5, ..., 66517.5, 66518.5, 66519.5],
      dtype=float32)

In [79]:
# Index of observation time <= obs_time
idx_t = np.searchsorted(ts, obstime_mjd-0.5)

In [80]:
t0 = ts[idx_t].item()
t1 = ts[idx_t+1].item()
w0 = (t1 - obstime_mjd) / (t1 - t0 )
w1 = (obstime_mjd - t0) / (t1 - t0)
print(f't0:      ', t0)
print(f't1:      ', t1)
print(f'obstime, ', obstime_mjd)
print(f'w0: ', w0)
print(f'w1: ', w1)

t0:       58600.5
t1:       58601.5
obstime,  58600.5
w0:  1.0
w1:  0.0


In [81]:
# Interpolated asteroid positions as observation time
q0 = outputs['q'][:, idx_t]
q1 = outputs['q'][:, idx_t+1]
q = w0 * q0 + w1 * q1

In [82]:
# Predicted position and direction of ceres
ast_mse = q[idx_ast].flatten()
np.round(ast_mse,6)

array([-1.356567, -2.37266 ,  0.175377], dtype=float32)

In [83]:
earth_mse = get_earth_pos(obstime_mjd)

In [84]:
np.round(earth_mse, 6)

array([-8.07118e-01, -6.01307e-01,  3.10000e-05])

In [85]:
rel_mse = ast_mse - earth_mse
np.round(rel_mse, 6)

array([-0.549449, -1.771353,  0.175346])

In [86]:
u_mse2 = rel_mse / np.linalg.norm(rel_mse)
np.round(u_mse2, 6)

array([-0.294946, -0.950867,  0.094126])

In [87]:
# MSE direction via lookup of output field u
u0 = outputs['u'][:, idx_t]
u1 = outputs['u'][:, idx_t+1]
u = w0 * u0 + w1 * u1
u_mse = u[idx_ast].flatten()
np.round(u_mse, 6)

array([-0.291472, -0.951963,  0.093866], dtype=float32)

In [88]:
# Demonstrate that two methods of getting direction vector from MSE essentially identical
np.linalg.norm(u_mse - u_mse2)

0.0036519647165336048

In [89]:
print(f'Heliocentric Cartesian coordinates from MSE Integration:')
print('Ceres:   ', ast_mse)
print('Earth:   ', earth_mse)
print('Relative:', rel_mse)
print('Direction:', u_mse)

Heliocentric Cartesian coordinates from MSE Integration:
Ceres:    [-1.3565673  -2.3726602   0.17537652]
Earth:    [-8.07117964e-01 -6.01307376e-01  3.06978658e-05]
Relative: [-0.5494493  -1.77135278  0.17534582]
Direction: [-0.2914719  -0.9519628   0.09386639]


**Compare MSE Calculations to JPL Using RA / DEC**

In [90]:
ast_err = ast_mse - ast_jpl
ast_err_norm = np.linalg.norm(ast_err)
print('ast_err: ', ast_err)
print(f'norm ast_err = {ast_err_norm:5.2e}')

ast_err:  [-4.74329871e-03  6.85181370e-03 -2.80875070e-05]
norm ast_err = 8.33e-03


In [91]:
earth_err = earth_mse - earth_jpl
earth_err_norm = np.linalg.norm(earth_err)
print('earth_err: ', earth_err)
print(f'norm earth_err = {earth_err_norm:5.2e}')

earth_err:  [-8.03812650e-06  1.11431126e-05  5.13274696e-10]
norm earth_err = 1.37e-05


In [92]:
rel_err = rel_mse - rel_jpl
rel_err_norm = np.linalg.norm(rel_err)
print('rel_err: ', rel_err)
print(f'norm rel_err = {rel_err_norm:5.2e}')

rel_err:  [-4.73526059e-03  6.84067059e-03 -2.80880202e-05]
norm rel_err = 8.32e-03


In [93]:
u_err = u_mse - u_jpl
u_err_norm = np.linalg.norm(u_err)
print('direction_err: ', u_err)
print(f'norm direction_err = {u_err_norm:5.2e}')
print(f'in degrees = {np.rad2deg(u_err_norm):6.4f}')

direction_err:  [ 1.30180860e-04 -4.15048835e-05 -1.66303317e-05]
norm direction_err = 1.38e-04
in degrees = 0.0079


**Compare MSE Calculations to JPL Using Heliocentric Long/Lat**

In [ ]:
ast_err2 = ast_mse - ast_jpl2
ast_err_norm2 = np.linalg.norm(ast_err2)
print('ast_err2: ', ast_err2)
print(f'norm ast_err2 = {ast_err_norm2:5.2e}')

In [ ]:
earth_err2 = earth_mse - earth_jpl2
earth_err_norm2 = np.linalg.norm(earth_err2)
print('earth_err2: ', earth_err2)
print(f'norm earth_err2 = {earth_err_norm2:5.2e}')

In [ ]:
rel_err2 = rel_mse - rel_jpl2
rel_err2_norm = np.linalg.norm(rel_err2)
print('rel_err2: ', rel_err2)
print(f'norm rel_err2 = {rel_err2_norm:5.2e}')

Errors using heliocentric long / lat are slightly higher than using direct RA / DEC transformation.